In [1]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import h5py

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
BATCH_SIZE = 32

Num GPUs Available:  1


In [2]:
# A memory-mapped array is kept on disk. However, it can be accessed and sliced like any ndarray.
# Memory mapping is especially useful for accessing small fragments of large files without reading
# the entire file into memory.

## only accepts .npy files
def generator(feature_name,label_name,batch_size):
    
    hf = h5py.File('/home/sorozco0612/dev/alan_bot/data_processing/road_data/data.h5', 'r')
    
    x = hf[feature_name]
    y = hf[label_name]
    
    index = 0
    while True:
        
        # batch has not met the end of the data
        if (index + batch_size < x.shape[0]):
            features = x[index:index+batch_size]
            labels = y[index:index+batch_size]
            index += batch_size
        else:
            # batch size will be smaller than the rest on last iteration
            features = x[index:]
            labels = y[index:]
            index = 0
        
        yield (features,labels)
        

train_generator = generator('x_train', 'y_train',BATCH_SIZE)
test_generator = generator('x_test','y_test',BATCH_SIZE)


In [3]:
model = models.Sequential()
model.add(layers.Conv2D(32,(3,3),activation='relu', input_shape=(432,614,3)))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.MaxPool2D((2,2)))

model.add(layers.Flatten())
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(2))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 430, 612, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 215, 306, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 213, 304, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 106, 152, 64)      0         
_________________________________________________________________
flatten (Flatten)            (None, 1031168)           0         
_________________________________________________________________
dense (Dense)                (None, 64)                65994816  
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 1

In [4]:
model.compile(optimizer='adam',
              loss = tf.keras.losses.MeanSquaredError(),
              metrics=['mean_squared_error'])

In [ ]:
# Include the epoch in the file name (uses `str.format`)
#checkpoint_path = "training/cp-{epoch:04d}.ckpt"
#checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights every 5 epochs
#cp_callback = tf.keras.callbacks.ModelCheckpoint(
 #   filepath=checkpoint_path, 
 #   verbose=1, 
 #   save_weights_only=True,
 #   save_freq=5*batch_size)

# Save the weights using the `checkpoint_path` format
#model.save_weights(checkpoint_path.format(epoch=0))

# Train model with new callback
#fit = model.fit(x_train,
 #               y_train,
  #              epochs=50, 
  #              #callbacks=[cp_callback], 
  #              verbose=2,
  #              validation_data=(x_test,y_test))
fit = model.fit(x=train_generator,
                steps_per_epoch= 12640 // BATCH_SIZE,
                #callbacks=[cp_callback], 
                validation_steps= 4214 // BATCH_SIZE,
                validation_data=test_generator,
                verbose=1,
                epochs=50)

33
Epoch 1/50
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33

In [ ]:
print(model.predict(x_test[10:15]))
print(y_test[10:15])

In [ ]:
print("Evaluate on test data")
results = model.evaluate(x_test,y_test, batch_size=128)
print("test loss, test acc:", results)

In [ ]:
## Save the model as a tflite version

# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('/home/sorozco0612/dev/alan_bot/model_creation/alan_bot_model.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
## save the model regularly
model.save('/home/sorozco0612/dev/alan_bot/model_creation/alan_bot_model')

In [ ]:
## may not need this
## convert keras model to ConcreteFunction, but why??
full_model = tf.function(lambda x: model(x))
full_model = full_model.get_concrete_function(tf.TensorSpec(model.inputs[0].shape,model.inputs[0].dtype))

## get frrozen graph definition (convert variables to constants)
frozen_func = convert_variables_to_constants_v2(full_model)
frozen_func.graph.as_graph_def()

In [ ]:
## may not need this
# inspect layers operations inside frozen graph
layers = [op.name for op in frozen_func.graph.get_operations()]
print("-" * 60)
print("Frozen model layers: ")
for layer in layers:
    print(layer)
print("-" * 60)
print("Frozen model inputs: ")
print(frozen_func.inputs)
print("Frozen model outputs: ")
print(frozen_func.outputs)

In [ ]:
## may not need this
tf.io.write_graph(graph_or_graph_def= frozen_func.graph,
                  logdir='',
                  name="alan_bot_model_frozen.pb",
                  as_text= False)

tf.io.write_graph(graph_or_graph_def= frozen_func.graph,
                  logdir='',
                  name="alan_bot_model_frozen.pbtxt",
                  as_text= True)

In [ ]:
## may not need this
image = np.zeros((432, 614, 3), np.uint8)
image_tensor = tf.convert_to_tensor(image, dtype=tf.float32)
image_tensor = tf.expand_dims(image_tensor, 0)

print(frozen_func(image_tensor)[0].numpy())

In [ ]:
import tf2onnx
from  tensorrt import OnnxParser

In [ ]:
## convert saved model to ONNX
!python3 -m tf2onnx.convert --saved-model "/home/sorozco0612/dev/alan_bot/model_creation/alan_bot_model" --output "alan_bot_model_onnx.onnx"

In [ ]:
import tensorrt as trt

In [ ]:
## create logger
trt_logger = trt.Logger(trt.Logger.INFO)

builder = trt.Builder(trt_logger)
builder.max_batch_size = 1
builder.max_workspace_size = 1 << 32
builder.fp16_mode = True
builder.int8_mode = True
#network_creation_flag = 1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_PRECISION)
network_creation_flag = 1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)

In [ ]:
network = builder.create_network(network_creation_flag)
profile = builder.create_optimization_profile()
config = builder.create_builder_config()

In [ ]:
profile.set_shape("conv2d_input:0",(1,432,614,3),(1,432,614,3),(1,432,614,3))
config.add_optimization_profile(profile)

In [ ]:
parser = trt.OnnxParser(network,trt_logger)

In [ ]:
parser.parse_from_file("alan_bot_model_onnx.onnx")

In [ ]:
engine = builder.build_engine(network,config)

In [ ]:
print(engine.num_layers)

In [ ]:
## save engine
with open("engine.plan", "wb") as f:
    f.write(engine.serialize())

In [ ]:
## Load engine
trt_logger = trt.Logger(trt.Logger.INFO)
runtime = trt.Runtime(trt_logger)
with open("engine.plan", "rb") as f:
    engine = runtime.deserialize_cuda_engine(f.read())

In [ ]:
import pycuda.driver as cuda
import pycuda.autoinit # For automatic creation and cleanup of CUDA context